In [10]:
import os
import hardposit as hp
import softposit as sp
import fixed2float as fi
from pathlib import Path
import re
import ast

In [11]:
import fixed2float
fixed2float.__version__

'4.0.1'

In [12]:
N, ES = 8, 0

In [13]:
with open(Path("../tb_ppu.log"), 'r') as f:
  log = f.read()
print(log)

(0x24, 0x81, 0xffffffdc02000000, 0x82)
(0x09, 0x63, 0xffffffdc57800000, 0x82)
(0x0d, 0x8d, 0xffffffdb39800000, 0x82)
(0x65, 0x12, 0xffffffdbf6800000, 0x82)
(0x01, 0x0d, 0xffffffdbf7500000, 0x82)
(0x76, 0x3d, 0xffffffe2a3500000, 0x82)
(0xed, 0x8c, 0xffffffe46b500000, 0x83)
(0xf9, 0xc6, 0xffffffe484b00000, 0x83)
(0xc5, 0xaa, 0xffffffe612f00000, 0x83)
(0xe5, 0x77, 0xffffffe2e8f00000, 0x82)
(0x12, 0x8f, 0xffffffe1a4f00000, 0x82)
(0xf2, 0xce, 0xffffffe1d0b00000, 0x82)
(0xe8, 0xc5, 0xffffffe229300000, 0x82)
(0x5c, 0xbd, 0xffffffe01c300000, 0x82)
(0x2d, 0x65, 0xffffffe1f4b00000, 0x82)
(0x63, 0x0a, 0xffffffe253b00000, 0x82)
(0x80, 0x20, 0xffffffe252b00000, 0x80)
(0xaa, 0x9d, 0xffffffe654b00000, 0x83)
(0x96, 0x13, 0xffffffe55db00000, 0x83)
(0x0d, 0x53, 0xffffffe5b0900000, 0x83)
(0x6b, 0xd5, 0xffffffe36c100000, 0x82)
(0x02, 0xae, 0xffffffe35f900000, 0x82)
(0x1d, 0xcf, 0xffffffe306c00000, 0x82)
(0x23, 0x0a, 0xffffffe31ca00000, 0x82)
(0xca, 0x3c, 0xffffffe252200000, 0x82)
(0xf2, 0x8a, 0xffffffe3da

In [14]:
log.splitlines()[0]

'(0x24, 0x81, 0xffffffdc02000000, 0x82)'

In [15]:
lst = list(map(lambda l: ast.literal_eval(l), log.splitlines()))

In [16]:
operand1_i, operand2_i, fixed_o, result_o = list(zip(*lst)) 

In [17]:
operand1_i, operand2_i, fixed_o, result_o

((36,
  9,
  13,
  101,
  1,
  118,
  237,
  249,
  197,
  229,
  18,
  242,
  232,
  92,
  45,
  99,
  128,
  170,
  150,
  13,
  107,
  2,
  29,
  35,
  202,
  242,
  65,
  120,
  235,
  198,
  188,
  11,
  133,
  59,
  126,
  241,
  98,
  159,
  248,
  159),
 (129,
  99,
  141,
  18,
  13,
  61,
  140,
  198,
  170,
  119,
  143,
  206,
  197,
  189,
  101,
  10,
  32,
  157,
  19,
  83,
  213,
  174,
  207,
  10,
  60,
  138,
  216,
  137,
  182,
  174,
  42,
  113,
  79,
  58,
  21,
  217,
  76,
  143,
  183,
  92),
 (18446743919124283392,
  18446743920558735360,
  18446743915760451584,
  18446743918931345408,
  18446743918944976896,
  18446743947600461824,
  18446743955250872320,
  18446743955676594176,
  18446743962358120448,
  18446743948768575488,
  18446743943332757504,
  18446743944066760704,
  18446743945551544320,
  18446743936743505920,
  18446743944670740480,
  18446743946264576000,
  18446743946247798784,
  18446743963461222400,
  18446743959317250048,
  184467439607076

In [18]:
acc = hp.from_bits(0, N, ES)
for (operand1, operand2) in zip(operand1_i, operand2_i):
  acc += (hp.from_bits(operand1, N, ES) * hp.from_bits(operand2, N, ES))

In [19]:
acc

10000000 P<8,0>(128) nan

In [20]:
M, B = 31, 64
fi.from_bits(fixed_o[-1], M, B)

1111111111111111111111111010100011010110010100000000000000000000 Fx<31,64>(18446743699347996672) -87.162841796875

In [21]:
hp.from_bits(result_o[-1], N, ES).eval()

-64.0

### test result using softposit

In [22]:
softposit_acc = [0] * len(operand1_i)

q = sp.quire16()

for i, (operand1, operand2) in enumerate(zip(operand1_i, operand2_i)):
  q.qma(
    sp.posit16(bits=operand1), sp.posit16(bits=operand2)
  )
  softposit_acc[i] = q.v

In [23]:
softposit_acc

[3.725290298461914e-09,
 3.725290298461914e-09,
 3.725290298461914e-09,
 3.725290298461914e-09,
 3.725290298461914e-09,
 3.725290298461914e-09,
 1.4901161193847656e-08,
 5.960464477539063e-08,
 5.960464477539063e-08,
 5.960464477539063e-08,
 5.960464477539063e-08,
 8.940696716308594e-08,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.1920928955078125e-07,
 1.7881393432617188e-07,
 1.7881393432617188e-07,
 1.7881393432617188e-07,
 1.7881393432617188e-07,
 1.7881393432617188e-07,
 1.7881393432617188e-07,
 1.7881393432617188e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.384185791015625e-07,
 2.980232238769531e-07,
 2.980232238769531e-07]

### FPPU hardware outputs

In [24]:
results_o = list(map(lambda bits: hp.from_bits(bits, N, ES).eval(), result_o))
results_o

[-32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -24.0,
 -24.0,
 -24.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 nan,
 -24.0,
 -24.0,
 -24.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -32.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0,
 -64.0]

In [25]:
list(map(lambda val1, val2: val1 == val2, map(str, results_o), map(str, softposit_acc)))

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [26]:
assert all(_) == True
print("Congratulation, it works.")

AssertionError: 